In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json
from pprint import pprint

from src.supabase_client import get_user
from tests.test_full_week import walk_through_week


# Test: Full Week

In [ ]:
# mock_user = get_user(os.environ["JAREDS_ATHLETE_ID"])
# response = walk_through_week(mock_user, "2024-09-01")

# for training_week in response:
#     print(training_week.total_mileage)

# Evals: New Training Week

In [ ]:
path = 'tests/artifacts/new_training_week_errors'
files = os.listdir(path)

for file in sorted(files):
    print(f"Loading {file}")
    with open(os.path.join(path, file), "r") as f:
        results = json.load(f)
    print(sum(result['error'] for result in results) / len(results))

In [ ]:
for result in results:
    print(result['user']['email'])
    print("target:", result['recommended_weekly_mileage'])
    print("actual:", result['generated_weekly_mileage'])
    print("error:", result['error'])
    print(result['planning'])
    print("." * 100)